In [88]:
s = "def greeting(name: str) -> str: \
    return 'Hello ' + name "

In [89]:
import astor,ast

In [90]:
a = ast.parse(s)

In [91]:
#!pipenv install typed-ast

In [92]:
from typed_ast import ast3

In [93]:
a = ast3.parse(s)

In [94]:
a

In [152]:
a.body[0].args.args[0].arg

'name'

In [157]:
hypothesis_mappings = {"str": "text())"}

def make_test(a):
    for i in a.body:
        name = i.name
        r = i.returns.id
        args_names = [x.arg for x in i.args.args]
        args_names = ", ".join(args_names)
        args_annotation = [ x.annotation.id for x in i.args.args]
        hypothesis_map = [find_mapping(arg) for arg in args_annotation]
        hypothesis_map =", ".join(hypothesis_map)
        decorator = "@given(" 
        t = decorator+hypothesis_map+")"+"\n"
        
        t += f'def test_{name}({args_names}):\n'
        t +=f'    assert type({name}({args_names})) is {r}'
        return t
def find_mapping(arg):
    if arg in hypothesis_mappings.keys():
        return hypothesis_mappings[arg]
    elif arg in build_dict.keys():
        return build_dict[arg]

In [158]:
print(make_test(a))

@given(text()))
def test_greeting(name):
    assert type(greeting(name)) is str


In [10]:
#!pipenv install hypothesis

In [11]:
from hypothesis import given
from hypothesis.strategies import text,builds,lists

def greeting(name: str) -> str: 
    return 'Hello ' + name 
@given(text())
def test_greeting(s):
    assert type(greeting(s)) is str

In [12]:
test_greeting()

In [13]:
from collections import namedtuple

In [14]:
from typing import List
class DataPoint:
    def __init__(self,x:int,y:int):
        self.x = x
        self.y =y

class DataSet:
    def __init__(self,label:str,data:List[DataPoint],backgroundColor:str):
        self.label=label
        self.data=data
        self.backgroundColor = backgroundColor

class Data:
    def __init__(self,datasets:List[DataSet]=None):
        self.datasets=datasets

In [15]:
data1 = Data([DataSet(label="label1",data=[DataPoint(1,2),DataPoint(3,4)],backgroundColor="00b8ff")])

In [16]:
from crawto.Charts.charts import tsne_plot,make_html

In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
from crawto.Charts.charts import tsne_plot,make_html
from hypothesis import given
from hypothesis.strategies import text,builds,lists,floats
#from crawto.Charts.chart_types import 

@given(builds(Data,lists(builds(DataSet,text(),lists(builds(DataPoint,floats(),floats())),text()))))
def test_tsne_chart(d):
    assert type(make_html(d)) is str

In [20]:
s=open("crawto/Charts/charts.py").read()
a = ast3.parse(s)

In [21]:
a.body

In [22]:
import typed_ast
type(a.body[0]) is typed_ast._ast3.ImportFrom

True

In [23]:
z =a.body[7]

In [24]:
z.args.args[0].annotation.id

AttributeError: 'Import' object has no attribute 'args'

In [ ]:
dir(a.body[4])

In [25]:
a.body[4].names[0].name,a.body[4].module,a.body[4].level

('Data', 'Charts.chart_type', 2)

In [26]:
import typing
ast_import_union = typing.Union[typed_ast._ast3.ImportFrom,typed_ast._ast3.Import]

In [27]:
for i in a.body[4].names:
    print(i.name)

Data


In [127]:
name_module_dict

{'Data': '/mnt/e/projects/crawto/crawto/Charts/chart_type.py'}

In [128]:
for i in name_module_dict:

Data


In [173]:
import typed_ast
import os
from typed_ast import ast3
import astor,ast
def make_tests_for_file(filename):
    s=open(filename).read()
    a = ast3.parse(s)
    name_module_dict = {}
    for i in a.body:
        map_relative_import_to_object(i,name_module_dict)
    full_build_dict = {}
    for i in name_module_dict:
        module = open(name_module_dict[i]).read()
        module = ast3.parse(module)
        build_dict = iterate_module_operate_class(module)
        if i in build_dict:
            full_build_dict[i] = build_dict[i]
    
    t = make_test(a, full_build_dict)
    return t
        
    
    
def map_relative_import_to_object(ast_import,name_module_dict):
    if type(ast_import) is typed_ast._ast3.ImportFrom:
        if ast_import.level>0:
            module=ast_import.module
            for i in ast_import.names:
                m = module.split(".")[-1]
                name_module_dict[i.name] = [
                    i for i,j,k in os.walk(os.getcwd()) if "chart_type.py" in k][0]+"/"+m.replace(".","/")+".py"
    return name_module_dict
tested_things = [typed_ast._ast3.FunctionDef,typed_ast._ast3.ClassDef]
generic_types=['str','int','float']
hypothesis_dict = {
    'str':'text()',
    'int':'int()',
    'float':'float()',
    'List':'lists'
}
hypothesis_mappings = {"str": "text())"}

def iterate_module_operate_class(ast3_Module):
    
    body = ast3_Module.body
    classes = {}
    for i in body:
        if type(i) is typed_ast._ast3.ClassDef:
            p = parse_class(i)
            classes[i.name] = p
    class_dict = find_and_replace_customs
    return class_dict(classes)
            
def get_class_names(ast_Module):
    class_names = []
    for i in b.body:
        try:
            class_names.append(i.name)
        except:
            pass
    return class_names

def parse_class(ast3_ClassDef):
    for i in ast3_ClassDef.body:
        if i.name == '__init__':
            args = [a.value.id for a in i.body]
            d = {}
            for i in i.args.args: 
                if parse_annotations(i) is not None:
                    att, annotation = parse_annotations(i)
                    d[att]=annotation
    return d

def parse_annotations(ast3_ClassDef_args_args_i):
    if ast3_ClassDef_args_args_i.arg =="self":
        pass
    elif 'annotation' in dir(ast3_ClassDef_args_args_i):
        att = ast3_ClassDef_args_args_i.arg
        try:
            annotation = ast3_ClassDef_args_args_i.annotation.id
        except:
            annotation = ast3_ClassDef_args_args_i.annotation.value.id
            if 'slice' in dir(ast3_ClassDef_args_args_i.annotation):
                slice_id = ast3_ClassDef_args_args_i.annotation.slice.value.id
                annotation = (annotation,slice_id)
        return att,annotation
    
def find_and_replace_customs(class_dict):
    build_dict = {}
    for i in class_dict:
        tmp_dict = {}
        for x,y in zip(list(class_dict[i].keys()),list(class_dict[i].values())):
            if y in generic_types:
                tmp_dict[x] = hypothesis_dict[y]
            elif type(y) is tuple:
                if y[0] is 'List':
                    if y[1] in build_dict:
                        tmp_dict[x] = build_dict[y[1]]
        build_string = f'builds({i}' 
        for x,y in zip(list(tmp_dict.keys()),list(tmp_dict.values())):
            build_string +=f',{x}={y}'
        build_string += ")"
        build_dict[i] = build_string
    return build_dict

def make_test(a,full_build_dict):
    tt = []
    for i in a.body:
        if type(i) in tested_things:
            if 'returns' in dir(i):
                if 'id' in dir(i.returns):
                    name = i.name
                    r = i.returns.id
                    if r in hypothesis_mappings.keys():
                        args_names = [x.arg for x in i.args.args]
                        args_names = ", ".join(args_names)
                        args_annotation = [ x.annotation.id for x in i.args.args]
                        hypothesis_map = [find_mapping(arg,full_build_dict) for arg in args_annotation]
                        hypothesis_map =", ".join(hypothesis_map)
                        decorator = "@given(" 
                        t = decorator+hypothesis_map+")"+"\n"
                        t += f'def test_{name}({args_names}):\n'
                        t +=f'    assert type({name}({args_names})) is {r}'
                        tt.append(t)
    return tt

def find_mapping(arg,full_build_dict):
    if arg in hypothesis_mappings.keys():
        return hypothesis_mappings[arg]
    elif arg in full_build_dict.keys():
        return full_build_dict[arg]

In [176]:
print(make_tests_for_file("crawto/Charts/charts.py")[0])

@given(builds(Data,datasets=builds(DataSet,label=text(),data=builds(DataPoint,x=float(),y=float()),backgroundColor=text())))
def test_make_html(data):
    assert type(make_html(data)) is str


In [83]:
'str' in hypothesis_mappings.keys()

True

In [178]:
from hypothesis import given
from hypothesis.strategies import text,builds,lists

@given(builds(Data,datasets=builds(DataSet,label=text(),data=builds(DataPoint,x=float(),y=float()),backgroundColor=text())))
def test_make_html(data):
    assert type(make_html(data)) is str

NameError: name 'Data' is not defined